# Refinitiv Data Platform Library for Python
## Content - Streaming Prices - How to update a DataFrame with streaming events

This notebook demonstrates how to use a StreamingPrice with events to update a Pandas DataFrame with real-time streaming data. Using a StreamingPrices that way allows your application to have at its own in memory representation (a Pandas DataFrame in this example) that is kept updated with the latest streaming values received from the Refinitiv Data Platform.  

#### Learn more

To learn more about the Refinitiv Data Platform Data Libraries just connect to the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/iam/login) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like [Quick Start guides](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start), [Tutorials](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/learning), [Documentation](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/docs) and much more.  

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.thomsonreuters.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

## Import the library and load credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [1]:
import refinitiv.dataplatform as rdp
import pandas as pd
from IPython.display import clear_output

%run ./credentials.ipynb

## Open the session of your choice

Depending on the access point your application uses to connect to the Refinitiv Data Platform, it needs to call one of the following functions to create and open a session to the platform. This session will become the default session used by all other function calls.

#### Either

Create and open a Desktop session to connect to the Refinitiv Data Platform pvia Eikon 4 or the Refinitiv Workspace.

In [2]:
rdp.open_desktop_session(APP_KEY)

#### or

Create and open a Platform session to connect directly to the Refinitiv Data Platform. 

In [7]:
rdp.open_platform_session(
    APP_KEY, 
    rdp.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
)

#### or

Create and open a Deployed Platform session to connect directly to a deployed enterprise platform (a.k.a.TREP). 

In [12]:
rdp.open_deployed_platform_session(
    APP_KEY,
    DEPLOYED_PLATFORM_HOST,
    DEPLOYED_PLATFORM_USER_NAME
)

## Define a callback to receive incoming events

In [13]:
global df

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = pd.DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)
    
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)


## Create a StreamingPrice and register the event callbacks

In [14]:
streaming_prices = rdp.StreamingPrices(
    universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
    fields   = ['BID','ASK'],
    on_complete = lambda streaming_prices : 
        create_dataframe(streaming_prices),
    on_update = lambda streaming_prices, instrument_name, fields : 
        update_dataframe(streaming_prices, instrument_name, fields)
)

## Open the StreamingPrice and wait for events

In [15]:
streaming_prices.open()

,BID,ASK
EUR=,1.1115,1.1118
GBP=,1.3037,1.304
JPY=,109.46,109.49
CAD=,1.3121,1.3122


## Close the StreamingPrice when done

In [16]:
streaming_prices.close()

<StreamState.Closed: 1>